Import potřebných nástrojů

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, ttest_ind

pd.options.mode.chained_assignment = None

Stažení dat

In [69]:
df = pd.read_pickle("accidents.pkl.gz")

# Hypotéza 1

V úvodní hypotéze se věnujeme tvrzení, že na dálnicích umírá stejné množství lidí jako na silnicích první třídy. Nulovou hypotézou je tvrzení, že umírá stejné množství lidí, alternativní, že umírá větší podíl lidí na silnicích první třídy

Nejprve vytvořím sloupec fatal, kde rozliším fatální nehody a nehody bez úmrtí a dálnice a silnice.

In [70]:
ddf = df[(df["p36"] == 0) | (df["p36"] == 1)]
ddf.loc[ddf["p36"] == 0, "typ"] = "Dálnice"
ddf.loc[ddf["p36"] == 1, "typ"] = "Silnice první třídy"
ddf.loc[ddf["p13a"] > 0, "fatal"] = "Fatální"
ddf.loc[ddf["p13a"] == 0, "fatal"] = "Bez úmrtí"

Následně vytvořím kontingenční tabulku s podíly fatálnosti na nehodách.

In [71]:
contingency = pd.crosstab(ddf["typ"], ddf["fatal"])
contingency

fatal,Bez úmrtí,Fatální
typ,,
Dálnice,24293,166
Silnice první třídy,78618,911


Pro zajímavost ještě zkusím normalizovanou kontingenčku.

In [72]:
contingencyPercentage = pd.crosstab(ddf["typ"], ddf["fatal"], normalize="index")
contingencyPercentage

fatal,Bez úmrtí,Fatální
typ,,
Dálnice,0.993213,0.006787
Silnice první třídy,0.988545,0.011455


Na kontingenční tabulce můžeme vidět, že na silnících první třídy (sloupec 1) umíral o něco větší podíl lidí než na dálnici. Otázkou je, zdali je tento rozdíl statisticky významný -- to ověříme testem chí kvadrátu.

In [73]:
chi2_contingency(contingency)

(39.314921530329016,
 3.6067450279444316e-10,
 1,
 array([[24205.67901104,   253.32098896],
        [78705.32098896,   823.67901104]]))

P-hodnota = 3.6067450279444316e-10. Naše nulová hypotéza říká, že hodnoty jsou různé. Pro tuto hypotézu zkoumáme hranici určenou $\alpha = 0.05$ podělenou dvěma (viz https://www.umat.fekt.vut.cz/~fusekmi/ipt/06.pdf a https://www.umat.fekt.vut.cz/~fusekmi/ipt/Tabulka_chi2.pdf) -- pro tuto hodnotu ($\chi ^2_{0.025}$) je hodnota z tabulky 0,001. Porovnáním určujeme, že na významnosti 95 % nemůžeme zamítnout nulovou hypotézu a tudíž nemůžeme říct, že by na silnicích první třídy umíralo více lidí. 

# Hypotéza 2

Ve druhé hypotéze tvrdíme, že při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi. Nulová hypotéza je taková, že jsou si rovny, a alternativní, že škody na vozidlech Škoda nižší.

Nejprve je nutné extrahovat data, která budou potřeba.

In [74]:
ddf2 = df[(df["p45a"] == 39) | (df["p45a"] == 2)]
ddf2.loc[ddf2["p45a"] == 39, "Značka"] = "Škoda"
ddf2.loc[ddf2["p45a"] == 2, "Značka"] = "Audi"


Nyní si vytvořím kontingenční tabulku.

In [75]:
contingencyCosts = pd.crosstab(index=ddf2["Značka"], columns="Škoda na vozidle", values=ddf2["p53"], aggfunc="mean")
contingencyCosts

col_0,Škoda na vozidle
Značka,
Audi,568.358934
Škoda,368.166026


Z kontingenční tabulky již poměrně zjevně vyplývá, že na škodovkách bývá menší škoda než na vozidlech značky Audi. To ještě ověříme vhodným testem.

Test musí ideálně být nezávislý (provnáváme dvě nezávislé hodnoty). Proto použiji párový nezávislý ttest.

In [76]:
skoda = ddf2.loc[ddf2["Značka"] == "Škoda", "p53"]
audi = ddf2.loc[ddf2["Značka"] == "Audi", "p53"]
tval, pval = ttest_ind(audi, skoda)
print("Hodnota T je:", tval)
print("Hodnota P je:", pval)


Hodnota T je: 39.12749433594028
Hodnota P je: 0.0


T hodnota nám říká, v jakém stupni volnosti studentova T-rozdělení máme hledat v tabulkách. Zaokrouhlením můžeme říci, že se jedná o 39. stupeň volnosti máme pro 95% hladinu významnosti hodnotu 1,685. Z hodnoty P (velmi malá) můžeme usoudit, že zamítáme nulovou hypotézu (hodnoty jsou stejné), a prohlásit, že škody na vozidlech značky Škoda jsou nižší.